In [1]:
import os
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models import DPINet
from data import PhysicsFleXDataset, collate_fn

from utils import count_parameters

In [2]:

parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--n_rollout', type=int, default=0)
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--env', default='')
parser.add_argument('--train_valid_ratio', type=float, default=0.9)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data')
parser.add_argument('--num_workers', type=int, default=10)
parser.add_argument('--gen_data', type=int, default=0)
parser.add_argument('--gen_stat', type=int, default=0)
parser.add_argument('--log_per_iter', type=int, default=1000)
parser.add_argument('--ckp_per_iter', type=int, default=10000)
parser.add_argument('--eval', type=int, default=0)
parser.add_argument('--verbose_data', type=int, default=1)
parser.add_argument('--verbose_model', type=int, default=1)

parser.add_argument('--n_instance', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

parser.add_argument('--n_epoch', type=int, default=1000)
parser.add_argument('--beta1', type=float, default=0.9)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--forward_times', type=int, default=2)

parser.add_argument('--resume_epoch', type=int, default=0)
parser.add_argument('--resume_iter', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)


_StoreAction(option_strings=['--relation_dim'], dest='relation_dim', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("--env FluidFall --gen_data 0".split())

In [4]:
args

Namespace(attr_dim=0, batch_size=1, beta1=0.9, ckp_per_iter=10000, dataf='data', dt=0.016666666666666666, env='FluidFall', eval=0, forward_times=2, gen_data=0, gen_stat=0, log_per_iter=1000, lr=0.0001, n_epoch=1000, n_his=0, n_instance=0, n_rollout=0, n_stages=0, nf_effect=200, nf_particle=200, nf_relation=300, num_workers=10, outf='files', position_dim=0, pstep=2, relation_dim=0, resume_epoch=0, resume_iter=0, shape_state_dim=14, state_dim=0, time_step=0, time_step_clip=0, train_valid_ratio=0.9, verbose_data=1, verbose_model=1)

In [5]:
phases_dict = dict()

args.n_rollout = 3000

# object states:
# [x, y, z, xdot, ydot, zdot]
args.state_dim = 6
args.position_dim = 3

# object attr:
# [fluid]
args.attr_dim = 1

# relation attr:
# [none]
args.relation_dim = 1

args.time_step = 121
args.time_step_clip = 5
args.n_instance = 1
args.n_stages = 1

args.neighbor_radius = 0.08

phases_dict["instance_idx"] = [0, 189]
phases_dict["root_num"] = [[]]
phases_dict["instance"] = ['fluid']
phases_dict["material"] = ['fluid']

args.outf = 'dump_FluidFall/' + args.outf

In [6]:
phases_dict

{'instance_idx': [0, 189],
 'root_num': [[]],
 'instance': ['fluid'],
 'material': ['fluid']}

In [7]:
args.outf = args.outf + '_' + args.env
args.dataf = 'data/' + args.dataf + '_' + args.env
print (args.dataf)
os.system('mkdir -p ' + args.outf)
os.system('mkdir -p ' + args.dataf)

data/data_FluidFall


0

In [8]:
# generate data
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}


In [9]:
datasets['train'].load_data(args.env)

In [10]:
datasets['valid'].load_data(args.env)

In [11]:
use_gpu = torch.cuda.is_available()

In [12]:
dataloaders = {x: torch.utils.data.DataLoader(
    datasets[x], batch_size=args.batch_size,
    shuffle=True if x == 'train' else False,
    num_workers=args.num_workers,
    collate_fn=collate_fn)
    for x in ['train', 'valid']}

In [13]:
len(datasets['train'])

324000

In [14]:
data = datasets['train'].__getitem__(1)
len(data)

positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5201, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.59221738 -0.23666683 -1.38101034]
[1.16937287 2.68465454 2.10023994]
[-0.08324916  0.95564153  0.35983307]
[0.70276658 1.01017992 0.86726888]
Velocities stats
(189, 3)
[ 8.58439481e-04 -1.14342875e+00  1.54762814e-03]
[0.81924527 0.2743061  0.79876307]
0 0 188 0 188


7

In [15]:
attr, state, rels, n_particles, n_shapes, instance_idx, label = data

In [16]:
print (attr.shape)

torch.Size([189, 1])


In [18]:
state
print(state.shape)

torch.Size([189, 6])


In [19]:
label

tensor([[-4.5844e-01, -1.0345e+00,  3.2358e-01],
        [ 1.7902e+00, -1.4487e+00,  3.9431e-02],
        [ 1.7902e+00, -1.4486e+00, -4.3107e-02],
        [-4.5865e-01, -1.0345e+00, -3.2735e-01],
        [ 1.8681e+00, -1.0576e+00, -2.1731e+00],
        [ 6.3302e-01, -1.6587e+00, -2.9972e-01],
        [ 6.3290e-01, -1.6587e+00,  2.9584e-01],
        [ 1.8683e+00, -1.0575e+00,  2.1696e+00],
        [-1.6891e+00, -1.5273e+00,  3.9515e+00],
        [-1.3358e+00, -1.5177e+00,  8.2070e-01],
        [-1.3360e+00, -1.5177e+00, -8.2467e-01],
        [-1.6892e+00, -1.5274e+00, -3.9556e+00],
        [-8.1523e-01, -1.5063e+00,  2.4789e-01],
        [-1.8015e-01, -1.3198e+00,  2.6544e-02],
        [-1.8029e-01, -1.3199e+00, -2.9875e-02],
        [-8.1519e-01, -1.5064e+00, -2.5077e-01],
        [-1.7412e-01, -1.1458e+00,  1.7861e-01],
        [ 5.5308e-01, -1.4363e+00, -8.4581e-01],
        [ 5.5301e-01, -1.4364e+00,  8.4197e-01],
        [-1.7422e-01, -1.1457e+00, -1.8246e-01],
        [ 6.5351e-01

In [20]:
rels

[[tensor([[   0,    0,    0,  ...,  188,  188,  188],
          [   0,    1,    2,  ..., 5198, 5199, 5200]])],
 [tensor([[   0,    1,    2,  ...,  186,  187,  188],
          [   0,    1,    2,  ..., 5198, 5199, 5200]])],
 [tensor([1., 1., 1.,  ..., 1., 1., 1.])],
 [tensor([[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]])],
 [array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
          26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
          39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
          52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
          65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
          91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
         104, 105, 106, 107, 

In [33]:
#Rr_idx
rels[0][0]

tensor([[   0,    0,    0,  ...,  188,  188,  188],
        [   0,    1,    2,  ..., 5198, 5199, 5200]])

In [29]:
rels[0][0][0]
# Rr_idxs: 每个relation所对应的receiver

tensor([  0,   0,   0,  ..., 188, 188, 188])

In [30]:
rels[0][0][1]
# Rr_idxs: 每个relation的编号

tensor([   0,    1,    2,  ..., 5198, 5199, 5200])

In [32]:
#Rs_idx
rels[1][0]

tensor([[   0,    1,    2,  ...,  186,  187,  188],
        [   0,    1,    2,  ..., 5198, 5199, 5200]])

In [42]:
#values
print (rels[2][0].shape)
rels[2][0]

torch.Size([5201])


tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [43]:
#Ras
print (rels[3][0].shape)
rels[3][0]

torch.Size([5201, 1])


tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [44]:
#node_r_idx
rels[4]

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [45]:
#node_s_idx
rels[5]

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [46]:
#psteps
rels[6]

[2]

In [30]:
n_particles

189

In [31]:
n_shapes

0

In [32]:
instance_idx

[0, 189]

In [35]:
label.shape

torch.Size([189, 3])

In [23]:
Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

In [40]:
len(rels[0])

1

In [38]:
Rr, Rs = [], []
for j in range(len(rels[0])):
    Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
    Rr.append(torch.sparse.FloatTensor(
        Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
    Rs.append(torch.sparse.FloatTensor(
        Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

In [44]:
torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])

torch.Size([189, 5201])

In [56]:
values.unique()

tensor([1.])

In [52]:
rels[0][0]

tensor([[   0,    0,    0,  ...,  188,  188,  188],
        [   0,    1,    2,  ..., 5198, 5199, 5200]])

In [51]:
Rr[0][0]

tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
       size=(5201,), nnz=25, layout=torch.sparse_coo)

In [39]:
args.forward_times = 1

In [45]:
# define propagation network
model = DPINet(args, datasets['train'].stat, phases_dict, residual=True, use_gpu=use_gpu)

print("Number of parameters: %d" % count_parameters(model))

if args.resume_epoch > 0 or args.resume_iter > 0:
    model_path = os.path.join(args.outf, 'net_epoch_%d_iter_%d.pth' % (args.resume_epoch, args.resume_iter))
    print("Loading saved ckp from %s" % model_path)
    model.load_state_dict(torch.load(model_path))

# criterion
criterionMSE = nn.MSELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=3, verbose=True)

if use_gpu:
    model = model.cuda()
    criterionMSE = criterionMSE.cuda()

st_epoch = args.resume_epoch if args.resume_epoch > 0 else 0
best_valid_loss = np.inf

Number of parameters: 615210


In [46]:

for epoch in range(st_epoch, args.n_epoch):

    phases = ['train', 'valid'] if args.eval == 0 else ['valid']
    for phase in phases:

        model.train(phase=='train')

        losses = 0.
        for i, data in enumerate(dataloaders[phase]):
   #         print ('i:',i)

            attr, state, rels, n_particles, n_shapes, instance_idx, label = data
            Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

            Rr, Rs = [], []
            for j in range(len(rels[0])):
                Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
                Rr.append(torch.sparse.FloatTensor(
                    Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
                Rs.append(torch.sparse.FloatTensor(
                    Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

            data = [attr, state, Rr, Rs, Ra, label]

            with torch.set_grad_enabled(phase=='train'):
                if use_gpu:
                    for d in range(len(data)):
                        if type(data[d]) == list:
                            for t in range(len(data[d])):
                                data[d][t] = Variable(data[d][t].cuda())
                        else:
                            data[d] = Variable(data[d].cuda())
                else:
                    for d in range(len(data)):
                        if type(data[d]) == list:
                            for t in range(len(data[d])):
                                data[d][t] = Variable(data[d][t])
                        else:
                            data[d] = Variable(data[d])

                attr, state, Rr, Rs, Ra, label = data

                # st_time = time.time()
                predicted = model(
                    attr, state, Rr, Rs, Ra, n_particles,
                    node_r_idx, node_s_idx, pstep,
                    instance_idx, phases_dict, args.verbose_model)
                # print('Time forward', time.time() - st_time)

           #     print(predicted.shape)
           #     print(label.shape)

            loss = criterionMSE(predicted, label)
       #     print ("loss",loss)
            losses += np.sqrt(loss.item())

            if phase == 'train':
                if i % args.forward_times == 0:
                    # update parameters every args.forward_times
              #      print ('update!')
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
              #      print ('done')


            if i % args.log_per_iter == 0:
                n_relations = 0
                for j in range(len(Ra)):
                    n_relations += Ra[j].size(0)
                print('%s [%d/%d][%d/%d] n_relations: %d, Loss: %.6f, Agg: %.6f' %
                      (phase, epoch, args.n_epoch, i, len(dataloaders[phase]),
                       n_relations, np.sqrt(loss.item()), losses / (i + 1)))

            if phase == 'train' and i > 0 and i % args.ckp_per_iter == 0:
                torch.save(model.state_dict(), '%s/net_epoch_%d_iter_%d.pth' % (args.outf, epoch, i))

        losses /= len(dataloaders[phase])
        print('%s [%d/%d] Loss: %.4f, Best valid: %.4f' %
              (phase, epoch, args.n_epoch, losses, best_valid_loss))

        if phase == 'valid':
            scheduler.step(losses)
            if(losses < best_valid_loss):
                best_valid_loss = losses
                torch.save(model.state_dict(), '%s/net_best.pth' % (args.outf))



train [0/1000][0/324000] n_relations: 5275, Loss: 0.894419, Agg: 0.894419
train [0/1000][1000/324000] n_relations: 5177, Loss: 0.807423, Agg: 0.418527
train [0/1000][2000/324000] n_relations: 5275, Loss: 0.648772, Agg: 0.394608
train [0/1000][3000/324000] n_relations: 5149, Loss: 0.349640, Agg: 0.382588
train [0/1000][4000/324000] n_relations: 5293, Loss: 0.561562, Agg: 0.373696
train [0/1000][5000/324000] n_relations: 5129, Loss: 0.161549, Agg: 0.372062
train [0/1000][6000/324000] n_relations: 5293, Loss: 0.368700, Agg: 0.370865
train [0/1000][7000/324000] n_relations: 5203, Loss: 0.249128, Agg: 0.369775
train [0/1000][8000/324000] n_relations: 5261, Loss: 0.518292, Agg: 0.368503
train [0/1000][9000/324000] n_relations: 5327, Loss: 0.550168, Agg: 0.366629
train [0/1000][10000/324000] n_relations: 5131, Loss: 0.160980, Agg: 0.364675
train [0/1000][11000/324000] n_relations: 5165, Loss: 0.154363, Agg: 0.363283
train [0/1000][12000/324000] n_relations: 5133, Loss: 0.184685, Agg: 0.362052

train [0/1000][106000/324000] n_relations: 5331, Loss: 0.221968, Agg: 0.330234
train [0/1000][107000/324000] n_relations: 4953, Loss: 0.230561, Agg: 0.330213
train [0/1000][108000/324000] n_relations: 5223, Loss: 0.315992, Agg: 0.330122
train [0/1000][109000/324000] n_relations: 5001, Loss: 0.246634, Agg: 0.329959
train [0/1000][110000/324000] n_relations: 5225, Loss: 0.637329, Agg: 0.329849
train [0/1000][111000/324000] n_relations: 5341, Loss: 0.483210, Agg: 0.329683
train [0/1000][112000/324000] n_relations: 5133, Loss: 0.133223, Agg: 0.329421
train [0/1000][113000/324000] n_relations: 5357, Loss: 0.314910, Agg: 0.329191
train [0/1000][114000/324000] n_relations: 5249, Loss: 0.361861, Agg: 0.329063
train [0/1000][115000/324000] n_relations: 5141, Loss: 0.108058, Agg: 0.328927
train [0/1000][116000/324000] n_relations: 5119, Loss: 0.165460, Agg: 0.328789
train [0/1000][117000/324000] n_relations: 5231, Loss: 0.181453, Agg: 0.328617
train [0/1000][118000/324000] n_relations: 5225, Los

train [0/1000][210000/324000] n_relations: 5079, Loss: 0.304609, Agg: 0.318248
train [0/1000][211000/324000] n_relations: 4937, Loss: 0.184369, Agg: 0.318114
train [0/1000][212000/324000] n_relations: 5127, Loss: 0.152193, Agg: 0.318013
train [0/1000][213000/324000] n_relations: 5189, Loss: 0.251668, Agg: 0.317883
train [0/1000][214000/324000] n_relations: 5147, Loss: 0.186243, Agg: 0.317765
train [0/1000][215000/324000] n_relations: 5171, Loss: 0.092183, Agg: 0.317697
train [0/1000][216000/324000] n_relations: 5195, Loss: 0.165975, Agg: 0.317625
train [0/1000][217000/324000] n_relations: 5335, Loss: 0.469914, Agg: 0.317510
train [0/1000][218000/324000] n_relations: 5155, Loss: 0.168188, Agg: 0.317413
train [0/1000][219000/324000] n_relations: 5099, Loss: 0.240894, Agg: 0.317311
train [0/1000][220000/324000] n_relations: 5195, Loss: 0.221486, Agg: 0.317239
train [0/1000][221000/324000] n_relations: 5131, Loss: 0.178543, Agg: 0.317133
train [0/1000][222000/324000] n_relations: 5385, Los

train [0/1000][314000/324000] n_relations: 5197, Loss: 0.110944, Agg: 0.310302
train [0/1000][315000/324000] n_relations: 5305, Loss: 0.309064, Agg: 0.310233
train [0/1000][316000/324000] n_relations: 5229, Loss: 0.115630, Agg: 0.310178
train [0/1000][317000/324000] n_relations: 5137, Loss: 0.228215, Agg: 0.310122
train [0/1000][318000/324000] n_relations: 5077, Loss: 0.136848, Agg: 0.310047
train [0/1000][319000/324000] n_relations: 5173, Loss: 0.163809, Agg: 0.309969
train [0/1000][320000/324000] n_relations: 5225, Loss: 0.181251, Agg: 0.309891
train [0/1000][321000/324000] n_relations: 5171, Loss: 0.179722, Agg: 0.309838
train [0/1000][322000/324000] n_relations: 5105, Loss: 0.598315, Agg: 0.309751
train [0/1000][323000/324000] n_relations: 5147, Loss: 0.104686, Agg: 0.309665
train [0/1000] Loss: 0.3096, Best valid: inf
valid [0/1000][0/36000] n_relations: 5207, Loss: 0.235901, Agg: 0.235901
valid [0/1000][1000/36000] n_relations: 5293, Loss: 0.360425, Agg: 0.284670
valid [0/1000][2

train [1/1000][59000/324000] n_relations: 5143, Loss: 0.145547, Agg: 0.284183
train [1/1000][60000/324000] n_relations: 4951, Loss: 0.174209, Agg: 0.284078
train [1/1000][61000/324000] n_relations: 5127, Loss: 0.150565, Agg: 0.284003
train [1/1000][62000/324000] n_relations: 5285, Loss: 0.163854, Agg: 0.284006
train [1/1000][63000/324000] n_relations: 5179, Loss: 0.170609, Agg: 0.283897
train [1/1000][64000/324000] n_relations: 5187, Loss: 0.109084, Agg: 0.283913
train [1/1000][65000/324000] n_relations: 5067, Loss: 0.343985, Agg: 0.283954
train [1/1000][66000/324000] n_relations: 5269, Loss: 0.572357, Agg: 0.284013
train [1/1000][67000/324000] n_relations: 5139, Loss: 0.092037, Agg: 0.283979
train [1/1000][68000/324000] n_relations: 5343, Loss: 0.406402, Agg: 0.283867
train [1/1000][69000/324000] n_relations: 5319, Loss: 0.194398, Agg: 0.283850
train [1/1000][70000/324000] n_relations: 5203, Loss: 0.173820, Agg: 0.283821
train [1/1000][71000/324000] n_relations: 5199, Loss: 0.189140, 

...

train [16/1000][67000/324000] n_relations: 5213, Loss: 0.161206, Agg: 0.234314
train [16/1000][68000/324000] n_relations: 5165, Loss: 0.157333, Agg: 0.234321
train [16/1000][69000/324000] n_relations: 4961, Loss: 0.165765, Agg: 0.234381
train [16/1000][70000/324000] n_relations: 5039, Loss: 0.139553, Agg: 0.234407
train [16/1000][71000/324000] n_relations: 5173, Loss: 0.144572, Agg: 0.234424
train [16/1000][72000/324000] n_relations: 5265, Loss: 0.292868, Agg: 0.234321
train [16/1000][73000/324000] n_relations: 4965, Loss: 0.168346, Agg: 0.234187
train [16/1000][74000/324000] n_relations: 5189, Loss: 0.094573, Agg: 0.234208
train [16/1000][75000/324000] n_relations: 5157, Loss: 0.155091, Agg: 0.234152
train [16/1000][76000/324000] n_relations: 5243, Loss: 0.303990, Agg: 0.234155
train [16/1000][77000/324000] n_relations: 5089, Loss: 0.124371, Agg: 0.234228
train [16/1000][78000/324000] n_relations: 5153, Loss: 0.092667, Agg: 0.234287
train [16/1000][79000/324000] n_relations: 5263, Los

train [16/1000][170000/324000] n_relations: 5149, Loss: 0.570867, Agg: 0.234380
train [16/1000][171000/324000] n_relations: 5155, Loss: 0.166314, Agg: 0.234398
train [16/1000][172000/324000] n_relations: 5297, Loss: 0.137934, Agg: 0.234381
train [16/1000][173000/324000] n_relations: 5147, Loss: 0.105967, Agg: 0.234327
train [16/1000][174000/324000] n_relations: 5299, Loss: 0.422462, Agg: 0.234284
train [16/1000][175000/324000] n_relations: 5139, Loss: 0.142593, Agg: 0.234277
train [16/1000][176000/324000] n_relations: 5147, Loss: 0.516106, Agg: 0.234284
train [16/1000][177000/324000] n_relations: 5383, Loss: 0.361548, Agg: 0.234299
train [16/1000][178000/324000] n_relations: 5129, Loss: 0.079771, Agg: 0.234323
train [16/1000][179000/324000] n_relations: 5153, Loss: 0.156435, Agg: 0.234380
train [16/1000][180000/324000] n_relations: 5311, Loss: 0.424852, Agg: 0.234385
train [16/1000][181000/324000] n_relations: 5129, Loss: 0.133945, Agg: 0.234360
train [16/1000][182000/324000] n_relatio

train [16/1000][273000/324000] n_relations: 5353, Loss: 0.456664, Agg: 0.234195
train [16/1000][274000/324000] n_relations: 5115, Loss: 0.093849, Agg: 0.234183
train [16/1000][275000/324000] n_relations: 5187, Loss: 0.173769, Agg: 0.234205
train [16/1000][276000/324000] n_relations: 5243, Loss: 0.284039, Agg: 0.234208
train [16/1000][277000/324000] n_relations: 5149, Loss: 0.135864, Agg: 0.234192
train [16/1000][278000/324000] n_relations: 5349, Loss: 0.367651, Agg: 0.234163
train [16/1000][279000/324000] n_relations: 4939, Loss: 0.162014, Agg: 0.234143
train [16/1000][280000/324000] n_relations: 5243, Loss: 0.200600, Agg: 0.234149
train [16/1000][281000/324000] n_relations: 5101, Loss: 0.154878, Agg: 0.234155
train [16/1000][282000/324000] n_relations: 5151, Loss: 0.156885, Agg: 0.234156
train [16/1000][283000/324000] n_relations: 5263, Loss: 0.373650, Agg: 0.234175
train [16/1000][284000/324000] n_relations: 5289, Loss: 0.323575, Agg: 0.234150
train [16/1000][285000/324000] n_relatio

train [17/1000][16000/324000] n_relations: 5333, Loss: 0.459211, Agg: 0.233673
train [17/1000][17000/324000] n_relations: 5275, Loss: 0.260118, Agg: 0.233828
train [17/1000][18000/324000] n_relations: 5309, Loss: 0.395366, Agg: 0.233791
train [17/1000][19000/324000] n_relations: 5141, Loss: 0.088054, Agg: 0.233780
train [17/1000][20000/324000] n_relations: 5159, Loss: 0.178192, Agg: 0.233808
train [17/1000][21000/324000] n_relations: 5251, Loss: 0.262954, Agg: 0.233718
train [17/1000][22000/324000] n_relations: 5263, Loss: 0.249461, Agg: 0.233844
train [17/1000][23000/324000] n_relations: 5263, Loss: 0.221547, Agg: 0.233560
train [17/1000][24000/324000] n_relations: 5273, Loss: 0.153904, Agg: 0.233521
train [17/1000][25000/324000] n_relations: 5119, Loss: 0.489411, Agg: 0.233831
train [17/1000][26000/324000] n_relations: 5161, Loss: 0.479518, Agg: 0.234035
train [17/1000][27000/324000] n_relations: 5297, Loss: 0.128057, Agg: 0.233954
train [17/1000][28000/324000] n_relations: 5043, Los

KeyboardInterrupt: 

In [28]:
args.forward_times

2

In [50]:
import os
os.listdir(args.outf)

['net_epoch_12_iter_310000.pth',
 'net_epoch_7_iter_180000.pth',
 'net_epoch_1_iter_250000.pth',
 'net_epoch_0_iter_240000.pth',
 'net_epoch_1_iter_150000.pth',
 'net_epoch_8_iter_200000.pth',
 'net_epoch_2_iter_40000.pth',
 'net_epoch_13_iter_30000.pth',
 'net_epoch_11_iter_210000.pth',
 'net_epoch_6_iter_320000.pth',
 'net_epoch_15_iter_70000.pth',
 'net_epoch_7_iter_270000.pth',
 'net_epoch_3_iter_150000.pth',
 'net_epoch_12_iter_320000.pth',
 'net_epoch_6_iter_220000.pth',
 'net_epoch_12_iter_200000.pth',
 'net_epoch_12_iter_100000.pth',
 'net_epoch_9_iter_70000.pth',
 'net_epoch_2_iter_250000.pth',
 'net_epoch_14_iter_120000.pth',
 'net_epoch_11_iter_120000.pth',
 'net_epoch_0_iter_300000.pth',
 'net_epoch_6_iter_190000.pth',
 'net_epoch_15_iter_270000.pth',
 'net_epoch_15_iter_10000.pth',
 'net_epoch_4_iter_120000.pth',
 'net_epoch_16_iter_290000.pth',
 'net_epoch_8_iter_120000.pth',
 'net_epoch_7_iter_30000.pth',
 'net_epoch_16_iter_190000.pth',
 'net_epoch_10_iter_150000.pth',


In [49]:
args.outf

'dump_FluidFall/files_FluidFall'